In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf
import re
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('../data/resep_tfidf_features.csv')

In [3]:
# Pisahkan judul resep dan fitur bahan
recipe_titles = df.iloc[:, 0].tolist()
X_features_df = df.iloc[:, 1:]
ingredient_list = X_features_df.columns.tolist()

In [4]:
# Konversi data resep menjadi TENSOR TensorFlow
recipe_matrix_binary = tf.constant(X_features_df.values > 0, dtype=tf.bool)
recipe_matrix_tfidf = tf.constant(X_features_df.values, dtype=tf.float32)

In [5]:
print(f"Data berhasil dimuat ke dalam TensorFlow Tensor.")
print(f"Jumlah resep: {recipe_matrix_binary.shape[0]}, Jumlah bahan unik: {recipe_matrix_binary.shape[1]}")
print("-" * 40)

Data berhasil dimuat ke dalam TensorFlow Tensor.
Jumlah resep: 172, Jumlah bahan unik: 348
----------------------------------------


In [6]:
def find_recipe_final(ingredients_input, titles, ingredients, matrix_binary, verbose=True):
    """
    Memproses input bahan menggunakan logika filtering dan skoring murni dengan TensorFlow.
    `verbose=False` untuk mematikan print saat evaluasi.
    """
    if verbose:
        print(f"\n--- MENCARI RESEP UNTUK: {ingredients_input} ---")

    tokenized_input = []
    for item in ingredients_input:
        words = re.split(r'\s+', str(item).lower())
        tokenized_input.extend(words)
    
    input_mask_np = np.isin(ingredients, tokenized_input)
    
    if not np.any(input_mask_np):
        return "NO_MATCH_INGREDIENTS" # Return kode untuk evaluasi
    
    if verbose:
        print(f"Bahan dikenali: {[ing for ing in tokenized_input if ing in ingredients]}")
    
    input_mask = tf.constant(input_mask_np, dtype=tf.bool)
    
    missing_ingredients_mask = tf.logical_and(tf.logical_not(matrix_binary), input_mask)
    missing_count_per_recipe = tf.reduce_sum(tf.cast(missing_ingredients_mask, tf.int32), axis=1)
    passing_recipes_mask = tf.equal(missing_count_per_recipe, 0)
    
    if not tf.reduce_any(passing_recipes_mask):
        return "NO_MATCH_RECIPE" # Return kode untuk evaluasi

    intersection = tf.reduce_sum(tf.cast(tf.logical_and(matrix_binary, input_mask), tf.float32), axis=1)
    total_ingredients_in_recipe = tf.reduce_sum(tf.cast(matrix_binary, tf.float32), axis=1)
    efficiency_score = intersection / (total_ingredients_in_recipe + 1e-6)
    
    final_scores = tf.where(passing_recipes_mask, efficiency_score, 0.0)
    
    best_recipe_index = tf.argmax(final_scores)
    best_score = final_scores[best_recipe_index]
    
    if best_score <= 0:
        return "NO_MATCH_SCORE" # Return kode untuk evaluasi
        
    matched_recipe_name = titles[best_recipe_index]
    
    # Untuk pemanggilan normal, kembalikan string yang informatif
    if verbose:
        return f"Resep yang paling sesuai adalah: **{matched_recipe_name}** (Skor Kecocokan: {best_score:.2f})"
    # Untuk evaluasi, kembalikan hanya judulnya
    else:
        return matched_recipe_name

In [7]:
def run_evaluation_suite(data_df, titles, ingredients, matrix_binary):
    """
    Menjalankan simulasi pengujian untuk menghasilkan metrik klasifikasi.
    """
    print("\n--- MEMULAI EVALUASI KINERJA MODEL ---")
    y_true = []
    y_pred = []
    
    for i in range(len(titles)):
        true_title = titles[i]
        
        # 1. Ambil bahan untuk resep saat ini
        recipe_ingredients_series = data_df.iloc[i, 1:]
        
        # 2. Buat skenario uji: gunakan 70% bahan terpenting
        important_ingredients = recipe_ingredients_series[recipe_ingredients_series > 0].sort_values(ascending=False)
        # Ambil 70% teratas, tapi minimal 2 bahan jika resepnya punya sedikit bahan
        num_to_keep = max(2, int(len(important_ingredients) * 0.7))
        test_ingredients = important_ingredients.head(num_to_keep).index.tolist()

        # 3. Dapatkan jawaban benar (y_true) dan prediksi model (y_pred)
        y_true.append(true_title)
        predicted_title = find_recipe_final(test_ingredients, titles, ingredients, matrix_binary, verbose=False)
        y_pred.append(predicted_title)
        
        # Print progress
        if (i + 1) % 40 == 0 or (i + 1) == len(titles):
            print(f"Evaluasi selesai untuk {i + 1}/{len(titles)} resep...")

    print("\n--- HASIL METRIK EVALUASI ---")
    
    # Menghitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Akurasi Keseluruhan: {accuracy:.2%}\n")

    # Membuat Classification Report
    # Note: 'zero_division=0' untuk menangani label yang tidak pernah diprediksi
    report = classification_report(y_true, y_pred, zero_division=0)
    print("Classification Report:")
    print(report)

    # Membuat Confusion Matrix (jika jumlah label < 30 agar terbaca)
    unique_labels = sorted(list(set(y_true + y_pred)))
    if len(unique_labels) < 30:
        cm = confusion_matrix(y_true, y_pred, labels=unique_labels)
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=unique_labels, yticklabels=unique_labels, cmap='Blues')
        plt.title('Confusion Matrix')
        plt.ylabel('Label Sebenarnya (True)')
        plt.xlabel('Label Prediksi')
        plt.show()
    else:
        print("\n(Confusion matrix tidak ditampilkan karena jumlah resep > 30, akan sulit dibaca)")

# Jalankan suite evaluasi lengkap
run_evaluation_suite(df, recipe_titles, ingredient_list, recipe_matrix_binary)



--- MEMULAI EVALUASI KINERJA MODEL ---
Evaluasi selesai untuk 40/172 resep...
Evaluasi selesai untuk 80/172 resep...
Evaluasi selesai untuk 120/172 resep...
Evaluasi selesai untuk 160/172 resep...
Evaluasi selesai untuk 172/172 resep...

--- HASIL METRIK EVALUASI ---
Akurasi Keseluruhan: 100.00%

Classification Report:
                                                                                                     precision    recall  f1-score   support

                                                                              10 minutes Mala Pasta       1.00      1.00      1.00         1
                                                           1186. Mochi miniatur kelapa versi simple       1.00      1.00      1.00         1
                                                                  19. Corndog sederhana mudah murah       1.00      1.00      1.00         1
                                                               20. Resep boba sederhana mudah murah       1.00   

In [8]:
def save_model_assets(filepath, titles, ingredients, matrix_binary_np):
    """
    Menyimpan semua aset yang diperlukan untuk deployment ke dalam satu file.
    """
    model_assets = {
        "recipe_titles": titles,
        "ingredient_list": ingredients,
        "recipe_matrix_binary": matrix_binary_np
    }
    with open(filepath, 'wb') as f:
        pickle.dump(model_assets, f)
    print(f"\nAset model berhasil disimpan ke: {filepath}")

In [9]:
save_model_assets("../model/recipe_model_assets.pkl", recipe_titles, ingredient_list, recipe_matrix_binary.numpy())


Aset model berhasil disimpan ke: ../model/recipe_model_assets.pkl


In [10]:
try:
    with open("../model/recipe_model_assets.pkl", 'rb') as f:
        loaded_assets = pickle.load(f)
    print("Aset model berhasil dimuat.")
except FileNotFoundError:
    print("Error: File 'recipe_model_assets.pkl' tidak ditemukan. Mohon jalankan skrip 'Buat_Model.py' terlebih dahulu.")
    exit()

# Ekstrak aset ke dalam variabel
recipe_titles = loaded_assets["recipe_titles"]
ingredient_list = loaded_assets["ingredient_list"]
recipe_matrix_binary = tf.constant(loaded_assets["recipe_matrix_binary"], dtype=tf.bool)

print(f"Total resep di dalam database: {len(recipe_titles)}")
print("-" * 40)

Aset model berhasil dimuat.
Total resep di dalam database: 172
----------------------------------------


In [11]:
def find_recipes(ingredients_input, num_recommendations=5, min_score_threshold=0.1):
    """
    Menemukan semua resep yang mengandung bahan input, diurutkan
    berdasarkan kecocokan tertinggi (Jaccard Similarity).
    """
    print(f"\n🍳 Mencari resep untuk: {ingredients_input}...")

    # 1. Tokenisasi dan persiapan input
    tokenized_input = []
    for item in ingredients_input:
        words = re.split(r'\s+', str(item).lower())
        tokenized_input.extend(words)
    
    input_mask_np = np.isin(ingredient_list, tokenized_input)
    
    recognized_ingredients = [ing for ing in tokenized_input if ing in ingredient_list]
    if not recognized_ingredients:
        return "Tidak ada bahan yang dikenali dalam sistem kami."
    
    input_mask = tf.constant(input_mask_np, dtype=tf.bool)
    
    # 2. Skoring Cerdas untuk SEMUA resep (tanpa filter awal)
    intersection = tf.reduce_sum(tf.cast(tf.logical_and(recipe_matrix_binary, input_mask), tf.float32), axis=1)
    union = tf.reduce_sum(tf.cast(tf.logical_or(recipe_matrix_binary, input_mask), tf.float32), axis=1)
    
    # Skor Jaccard dihitung untuk semua resep
    final_scores = intersection / (union + 1e-6) # Ditambah epsilon untuk menghindari pembagian dengan nol

    # 3. Cek apakah ada kecocokan sempurna (skor >= 0.999)
    perfect_match_indices = tf.where(final_scores >= 0.999).numpy().flatten()
    
    if len(perfect_match_indices) > 0:
        response = "**Resep Relevan Ditemukan!**\nBahan Anda cocok sempurna dengan resep berikut:\n"
        for idx in perfect_match_indices:
            response += f"- **{recipe_titles[idx]}**\n"
        return response
    
    # 4. Jika tidak ada yang sempurna, cari beberapa rekomendasi terbaik
    else:
        # Ambil top N rekomendasi dari semua skor
        top_k_scores, top_k_indices = tf.nn.top_k(final_scores, k=num_recommendations)
        
        recommendations = []
        for i in range(len(top_k_scores.numpy())):
            score = top_k_scores.numpy()[i]
            index = top_k_indices.numpy()[i]
            
            # Hanya tampilkan jika skor di atas ambang batas minimal
            if score >= min_score_threshold:
                recommendations.append((recipe_titles[index], score))
        
        if recommendations:
            response = "**Berikut Rekomendasi Resep yang Mengandung Bahan Anda:**\n(Diurutkan dari yang paling cocok)\n"
            for i, (title, score) in enumerate(recommendations):
                response += f"{i+1}. **{title}** (Skor Kecocokan: {score:.2f})\n"
            return response
        else:
            return f"Tidak ada resep yang cocok (Skor tertinggi di bawah ambang batas {min_score_threshold})."

In [12]:
bahan = ['mie instan goreng', 'telur', 'kaldu jamur']
hasil = find_recipes(bahan)
print(hasil)


🍳 Mencari resep untuk: ['mie instan goreng', 'telur', 'kaldu jamur']...
**Resep Relevan Ditemukan!**
Bahan Anda cocok sempurna dengan resep berikut:
- **Omelet Mie (instant goreng)**



In [13]:
bahan = ['mie', 'ayam']
hasil = find_recipes(bahan)
print(hasil)


🍳 Mencari resep untuk: ['mie', 'ayam']...
**Berikut Rekomendasi Resep yang Mengandung Bahan Anda:**
(Diurutkan dari yang paling cocok)
1. **Bubur Ayam Sat Set** (Skor Kecocokan: 0.33)
2. **Pizza mie instan** (Skor Kecocokan: 0.20)
3. **Mie goreng instan** (Skor Kecocokan: 0.20)
4. **Mie Kriuk (Mie Goreng Instan)** (Skor Kecocokan: 0.20)
5. **Omelet Mie (instant goreng)** (Skor Kecocokan: 0.17)

